In [11]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, datasets

In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')

In [3]:
EPOCHS = 30
BATCH_SIZE = 64

transform = transforms.Compose([transforms.ToTensor()])
trainset = datasets.FashionMNIST(root = './data/', train = True, download = True, transform = transform)
testset = datasets.FashionMNIST(root = './data/', train = False, download = True, transform = transform)
train_loader = torch.utils.data.DataLoader(dataset=trainset, batch_size=BATCH_SIZE)
test_loader = torch.utils.data.DataLoader(dataset=testset, batch_size=BATCH_SIZE)

C:\Users\ryu\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [9]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = x.view(-1, 784)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(x)
        return x
    

In [5]:
model = Net().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [6]:
def train(model, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

In [7]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [10]:
for epoch in range(1, EPOCHS+1):
    train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch, test_loss, test_accuracy))

[1] Test Loss: 0.5894, Accuracy: 79.10%
[2] Test Loss: 0.5540, Accuracy: 80.41%
[3] Test Loss: 0.5314, Accuracy: 81.05%
[4] Test Loss: 0.5149, Accuracy: 81.58%
[5] Test Loss: 0.5001, Accuracy: 82.08%
[6] Test Loss: 0.4893, Accuracy: 82.40%
[7] Test Loss: 0.4809, Accuracy: 82.68%
[8] Test Loss: 0.4724, Accuracy: 83.00%
[9] Test Loss: 0.4626, Accuracy: 83.37%
[10] Test Loss: 0.4544, Accuracy: 83.82%
[11] Test Loss: 0.4470, Accuracy: 84.10%
[12] Test Loss: 0.4403, Accuracy: 84.37%
[13] Test Loss: 0.4344, Accuracy: 84.62%
[14] Test Loss: 0.4280, Accuracy: 84.74%
[15] Test Loss: 0.4212, Accuracy: 84.94%
[16] Test Loss: 0.4152, Accuracy: 85.13%
[17] Test Loss: 0.4109, Accuracy: 85.30%
[18] Test Loss: 0.4063, Accuracy: 85.46%
[19] Test Loss: 0.4016, Accuracy: 85.68%
[20] Test Loss: 0.3992, Accuracy: 85.71%
[21] Test Loss: 0.3957, Accuracy: 85.80%
[22] Test Loss: 0.3922, Accuracy: 85.95%
[23] Test Loss: 0.3883, Accuracy: 86.08%
[24] Test Loss: 0.3851, Accuracy: 86.26%
[25] Test Loss: 0.3821, A